# Tiền xử lý

In [13]:
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
from torchvision import transforms
from utilities import extract_bubbles, append_to_file
from image_processing import getNails
from grid_info import getGridmatrix, getExtractsections
from fixed_coordinates import fixed_circle
import pytorch_lightning as pl
import os
from result_processsing import getSubmitResult

ModuleNotFoundError: No module named 'pandas'

In [682]:
img_path = "IMG_1581_iter_0.jpg"
coord_saver = "test.txt"


In [683]:
fixed_circle(img_path, coord_saver)

Centers of black squares (YOLOv8 format): [(0.9157427937915743, 0.928911456680419), (0.7707317073170732, 0.9247857822913361), (0.6283813747228382, 0.9216121866074262), (0.4878048780487805, 0.9190733100602984), (0.34767184035476717, 0.9174865122183434), (0.20709534368070953, 0.9165344335131704), (0.06607538802660753, 0.9152649952396065), (0.9152993348115299, 0.6708981275785465), (0.7725055432372505, 0.6686766105998095), (0.7015521064301552, 0.6677245318946367), (0.6310421286031042, 0.6667724531894637), (0.4909090909090909, 0.6651856553475087), (0.3507760532150776, 0.6642335766423357), (0.2811529933481153, 0.6635988575055538), (0.2106430155210643, 0.6632814979371628), (0.06873614190687362, 0.6620120596635989), (0.9170731707317074, 0.5445890193589337), (0.7033259423503326, 0.5420501428118057), (0.49223946784922396, 0.5401459854014599), (0.282039911308204, 0.5385591875595049), (0.06962305986696231, 0.536337670580768), (0.9206208425720621, 0.34306569343065696), (0.8310421286031042, 0.326880

In [684]:
import cv2
from train_utils import *
import matplotlib.pyplot as plt
inputs = []
targets = []
image = cv2.imread(img_path)
labels, coords = read_file_to_tensors(coord_saver)
for i, coord in enumerate(coords):
    x1, y1, x2, y2 = find_yolov8_square(image, coord)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv2.imwrite("/Users/phamminhtuan/Desktop/AIChallenge/avg_coords.jpg", image)

True

# Model

In [685]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseSeparableConv, self).__init__()
        # Depthwise convolution
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)

        # Pointwise convolution
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = torch.nn.functional.relu6(self.bn1(self.depthwise(x)))
        x = torch.nn.functional.relu6(self.bn2(self.pointwise(x)))
        return x

In [686]:
class MobileNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNet, self).__init__()

        def conv_bn(in_channels, out_channels, stride):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU6(inplace=True)
            )

        self.model = nn.Sequential(
            conv_bn(3, 32, 2),  # input size: 32x32x3, output size: 16x16x32
            DepthwiseSeparableConv(32, 64, 1),  # output size: 16x16x64
            DepthwiseSeparableConv(64, 128, 2), # output size: 8x8x128
            DepthwiseSeparableConv(128, 128, 1),# output size: 8x8x128
            DepthwiseSeparableConv(128, 256, 2),# output size: 4x4x256
            DepthwiseSeparableConv(256, 256, 1),# output size: 4x4x256
            DepthwiseSeparableConv(256, 512, 2),# output size: 2x2x512
            DepthwiseSeparableConv(512, 512, 1),# 5x repeated, output size: 2x2x512
            DepthwiseSeparableConv(512, 512, 1),
            DepthwiseSeparableConv(512, 512, 1),
            DepthwiseSeparableConv(512, 512, 1),
            DepthwiseSeparableConv(512, 512, 1),
            DepthwiseSeparableConv(512, 1024, 2),# output size: 1x1x1024
            DepthwiseSeparableConv(1024, 1024, 1),# output size: 1x1x1024
            nn.AdaptiveAvgPool2d(1), # output size: 1x1x1024
        )
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

# Khởi tạo mô hình
model = MobileNet(num_classes=2)  # Chuyển mô hình sang GPU nếu có



In [687]:
class CroatianFishClassifier(pl.LightningModule):
    def __init__(self, num_classes = 2):
        super(CroatianFishClassifier, self).__init__()
        self.model = model
        self.train_accuracies = []

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = torch.nn.functional.cross_entropy(logits, labels)
        acc = self.accuracy(logits, labels)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = torch.nn.functional.cross_entropy(logits, labels)
        acc = self.accuracy(logits, labels)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=5e-4)

    def accuracy(self, logits, labels):
        _, predicted = torch.max(logits.data, 1)
        correct = (predicted == labels).sum().item()
        total = labels.size(0)
        out = correct / total
        return out

    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = F.cross_entropy(logits, labels)
        acc = self.accuracy(logits, labels)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        return {'test_loss': loss, 'test_acc': acc}

In [688]:
# Kiểm tra checkpoint tốt nhất
best_checkpoint_path = "/Users/phamminhtuan/Desktop/AIChallenge/lightning_logs/version_8/checkpoints/best-checkpoint.ckpt"
if best_checkpoint_path:
    print(f"Loading best checkpoint from: {best_checkpoint_path}")
    
    # Load mô hình từ checkpoint (gọi trực tiếp trên class)
    model = CroatianFishClassifier.load_from_checkpoint(best_checkpoint_path)
    
    # Chuyển mô hình sang chế độ đánh giá
    model.eval()
    model.freeze()
else:
    print("No best checkpoint found!")


Loading best checkpoint from: /Users/phamminhtuan/Desktop/AIChallenge/lightning_logs/version_8/checkpoints/best-checkpoint.ckpt


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/lightning_fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


# Test argument

In [689]:
h, w = image.shape[:2]
print(h, w)

3151 2255


In [690]:
import math
def sort_to_convex_quadrilateral(dots):
    """
    Sorts a tuple of 4 dots into the order of a convex quadrilateral.

    :param dots: A tuple or list of 4 tuples, each representing (x, y) coordinates of a point
    :return: A list of 4 tuples sorted into a convex quadrilateral order
    """
    if len(dots) != 4:
        raise ValueError("Input must contain exactly 4 dots.")

    # Calculate the centroid of the points
    centroid_x = sum(dot[0] for dot in dots) / 4
    centroid_y = sum(dot[1] for dot in dots) / 4

    # Function to calculate the polar angle of a point relative to the centroid
    def polar_angle(dot):
        x, y = dot
        return math.atan2(y - centroid_y, x - centroid_x)

    # Sort the dots by their polar angle relative to the centroid
    sorted_dots = sorted(dots, key=polar_angle)
    return sorted_dots

In [691]:
input_data = "test.txt"

In [692]:
from shapely.geometry import Point, Polygon
from sklearn.cluster import DBSCAN
def getBubblesInClass(input_image_path, input_data, section_index, axis = 0, eps = 0.002):
        centers = image_processing.getNails(input_image_path)
        print("Length of the centers: ",centers)
        gridmatrix = grid_info.getGridmatrix(centers)
        gridsection = grid_info.getExtractsections(gridmatrix)
        dots = extract_bubbles(input_data)
        corners = gridsection[section_index[0]][section_index[1]]
        corners = sort_to_convex_quadrilateral(corners)
        if len(corners) != 4:
                raise ValueError("Exactly 4 corners are required to define a quadrilateral.")

                # Create a Polygon object using the corners
        polygon = Polygon(corners)

                # Filter dots that are inside the polygon (only checking x and y values)
        inside_dots = [dot for dot in dots if polygon.contains(Point(dot[1], dot[2]))]
        dots = inside_dots
        values = np.array([dot[axis + 1] for dot in dots]).reshape(-1, 1)
        if axis == 0:
            eps = 0.005

        # Apply DBSCAN clustering
        clustering = DBSCAN(eps=eps, min_samples=1).fit(values)

        # Group dots by their cluster labels
        batches = {}
        for label, dot in zip(clustering.labels_, dots):
            if label not in batches:
                batches[label] = []
            batches[label].append(dot)

        # Convert batches to a list of clusters
        clustered_batches = [
            sorted(batches[label], key=lambda dot: dot[1 if axis == 1 else 2])
            for label in sorted(batches.keys())
        ]
        return sorted(clustered_batches, key=lambda batch: batch[0][axis + 1])
      


In [693]:

sections = [
            {"name": "SBD", "section_index": (0, 0), "axis": 0, "eps": 0.002, "input_string": "SBD", "gap_string": 0},
            {"name": "MDT", "section_index": (0, 1), "axis": 0, "eps": 0.002, "input_string": "MDT", "gap_string": 0},
            {"name": "phan1_1", "section_index": (1, 0), "axis": 1, "eps": 0.002, "input_string": "1.", "gap_string": 0},
            {"name": "phan1_2", "section_index": (1, 1), "axis": 1, "eps": 0.002, "input_string": "1.", "gap_string": 10},
            {"name": "phan1_3", "section_index": (1, 2), "axis": 1, "eps": 0.002, "input_string": "1.", "gap_string": 20},
            {"name": "phan1_4", "section_index": (1, 3), "axis": 1, "eps": 0.002, "input_string": "1.", "gap_string": 30},
            {"name": "phan2_1", "section_index": (2, 0), "axis": 1, "eps": 0.002, "input_string": "2.1", "gap_string": "a"},
            {"name": "phan2_2", "section_index": (2, 1), "axis": 1, "eps": 0.002, "input_string": "2.2", "gap_string": "a"},
            {"name": "phan2_3", "section_index": (2, 2), "axis": 1, "eps": 0.002, "input_string": "2.3", "gap_string": "a"},
            {"name": "phan2_4", "section_index": (2, 3), "axis": 1, "eps": 0.002, "input_string": "2.4", "gap_string": "a"},
            {"name": "phan2_5", "section_index": (2, 4), "axis": 1, "eps": 0.002, "input_string": "2.5", "gap_string": "a"},
            {"name": "phan2_6", "section_index": (2, 5), "axis": 1, "eps": 0.002, "input_string": "2.6", "gap_string": "a"},
            {"name": "phan2_7", "section_index": (2, 6), "axis": 1, "eps": 0.002, "input_string": "2.7", "gap_string": "a"},
            {"name": "phan2_8", "section_index": (2, 7), "axis": 1, "eps": 0.002, "input_string": "2.8", "gap_string": "a"},
            {"name": "phan3_1", "section_index": (3, 0), "axis": 0, "eps": 0.002, "input_string": "3.1", "gap_string": "none"},
            {"name": "phan3_2", "section_index": (3, 1), "axis": 0, "eps": 0.002, "input_string": "3.2", "gap_string": "none"},
            {"name": "phan3_3", "section_index": (3, 2), "axis": 0, "eps": 0.002, "input_string": "3.3", "gap_string": "none"},
            {"name": "phan3_4", "section_index": (3, 3), "axis": 0, "eps": 0.002, "input_string": "3.4", "gap_string": "none"},
            {"name": "phan3_5", "section_index": (3, 4), "axis": 0, "eps": 0.002, "input_string": "3.5", "gap_string": "none"},
            {"name": "phan3_6", "section_index": (3, 5), "axis": 0, "eps": 0.002, "input_string": "3.6", "gap_string": "none"}
        ]

In [16]:
import torch
device = torch.device("mps")
# Your existing loop where you get the output from the model
folder_path = "/Users/phamminhtuan/Desktop/Trainning_SET/Images"
coord_saver = "test.txt"
result_txt_path = 'results_test_cnn.txt'
for filename in os.listdir(folder_path):
    with open("temp.txt", 'w') as file:
        pass
    if filename.startswith("IMG_1581"):
        input_image_path = os.path.join(folder_path, filename)
        input_image = cv2.imread(input_image_path)
        fixed_circle(input_image_path, coord_saver)
        for section in sections:
            Bubbles = getBubblesInClass(input_image_path, coord_saver, section["section_index"], axis=section["axis"], eps=section["eps"])
            for Class in Bubbles:
                    max_confidence = 0
                    best_coords = None   
                    for label in Class:
                        _, x1, y1, x2, y2 = label
                        coord = (x1, y1, x2, y2)
                        x1, y1, x2, y2 = find_yolov8_square(image, coord)
                        input = get_box(image, (x1, y1, x2, y2))
                        
                        transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Resize((32, 32))
                        ])
                        model.eval()
                        input = transform(input).to(device)
                        output = model(input.unsqueeze(0))
                        confidence_score = output[0][0].item()  # Access the first element, which is the confidence score

                        # Cập nhật bounding box có confidence cao nhất trong class
                        if confidence_score > max_confidence:
                            max_confidence = confidence_score
                            best_coords = (x1, y1, x2, y2)

                    if best_coords:
                        x1, y1, x2, y2 = best_coords
                        # Ghi tọa độ vào file txt
                        cv2.rectangle(input_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        with open("temp.txt", 'a') as f:     
                            f.write(f"0 {x1/w} {y1/h} {x2/w} {y2/h}\n")  # Ghi một dòng tọa độ
        getSubmitResult(input_image_path, input_data, result_txt_path)
cv2.imwrite("/Users/phamminhtuan/Desktop/AIChallenge/output1.jpg", input_image)         

            


ModuleNotFoundError: No module named 'torch'

In [604]:
from result_processsing import getSubmitResult
input_image_path = '/Users/phamminhtuan/Desktop/Trainning_SET/Images/IMG_1581_iter_0.jpg'
input_data = 'temp.txt'
result_txt_path = 'results_test_cnn.txt'
getSubmitResult(input_image_path, input_data, result_txt_path)

Processing SBD...
Processing MDT...
Processing phan1_1...
Processing phan1_2...
Processing phan1_3...
Processing phan1_4...
Processing phan2_1...
Processing phan2_2...
Processing phan2_3...
Processing phan2_4...
Processing phan2_5...
Processing phan2_6...
Processing phan2_7...
Processing phan2_8...
Processing phan3_1...
Processing phan3_2...
Processing phan3_3...
Processing phan3_4...
Processing phan3_5...
Processing phan3_6...


## Test the nails coordinate

In [17]:
import cv2
from utilities import extract_bubbles, append_to_file
import image_processing 
import grid_info 
from fixed_coordinates import fixed_circle
img_path = "D:\\4_2_AREAS\\Trainning_SET\\Images\\IMG_1581_iter_237.jpg"
input_image = cv2.imread(img_path)
coord_saver = "test.txt"
centers = image_processing.getNails(img_path)
print("The lens of the centers is: ",len(centers))
gridmatrix = grid_info.getGridmatrix(centers)
gridsection = grid_info.getExtractsections(gridmatrix)

# for center in centers:
#     x1, y1 = center
#     x1 = int(x1*w)
#     y1 = int(y1*h)
#     cv2.circle(input_image, (x1,y1),10, (0, 255, 0), 2)
# cv2.imwrite("/Users/phamminhtuan/Desktop/AIChallenge/avg_coords.jpg", input_image)

[ WARN:0@1524.032] global loadsave.cpp:241 findDecoder imread_('D:\4_2_AREAS\Trainning_SET\Images\IMG_1581_iter_237.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1524.033] global loadsave.cpp:241 findDecoder imread_('D:\4_2_AREAS\Trainning_SET\Images\IMG_1581_iter_237.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [15]:
gridsection

[[[(0.6886917960088692, 0.13011742304030466),
   (0.8337028824833703, 0.13075214217708664),
   (0.8310421286031042, 0.3268803554427166),
   (0.6864745011086475, 0.3252935576007617)],
  [(0.8337028824833703, 0.13075214217708664),
   (0.9254988913525499, 0.07457949857188194),
   (0.9206208425720621, 0.34306569343065696),
   (0.8310421286031042, 0.3268803554427166)],
  None,
  None,
  None,
  None,
  None,
  None],
 [[(0.07228381374722838, 0.33767058076801015),
   (0.282039911308204, 0.3386226594731831),
   (0.282039911308204, 0.5385591875595049),
   (0.06962305986696231, 0.536337670580768)],
  [(0.282039911308204, 0.3386226594731831),
   (0.49401330376940134, 0.34052681688352904),
   (0.49223946784922396, 0.5404633449698508),
   (0.282039911308204, 0.5385591875595049)],
  [(0.49401330376940134, 0.34052681688352904),
   (0.7068736141906874, 0.34052681688352904),
   (0.7033259423503326, 0.5420501428118057),
   (0.49223946784922396, 0.5404633449698508)],
  [(0.7068736141906874, 0.3405268168

In [12]:
for filename in os.listdir(folder_path):
    with open(coord_saver, 'w') as file:
        pass
    if filename.startswith("IMG_1581_iter_237"):
        input_image_path = os.path.join(folder_path, filename)
        centers = image_processing.getNails(input_image_path)
        for center in centers:
            x1, y1 = center
            x1 = int(x1*w)
            y1 = int(y1*h)
            cv2.circle(input_image, (x1,y1),10, (0, 255, 0), 2)
        cv2.imwrite("/Users/phamminhtuan/Desktop/AIChallenge/avg_coords.jpg", input_image)
        print(input_image_path, len(centers))
        
    

NameError: name 'os' is not defined

In [2]:
import os
from fixed_coordinates import *
from train_utils import *
from result_processsing import *
from model import Model
model = Model()
device = torch.device("mps")
coord_saver = "test1.txt"
result_txt_path = 'results_test_template.txt'
folder_path = "/Users/phamminhtuan/Downloads/testset1/images"
input_path = os.path.join(folder_path, "IMG_3967_iter_2.jpg")
fixed_circle(input_path, coord_saver)
labels, coords = read_file_to_tensors(coord_saver)
image = cv2.imread(input_path)
h, w = image.shape[:2]
for i, coord in enumerate(coords):
                    xy_coord = find_yolov8_square(image, coord)
                    x1, y1, x2, y2 = xy_coord
                    input = get_box(image, xy_coord)
                    transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Resize((32, 32))
                                ])
                    input = transform(input).to(device)
                    output = model(input.unsqueeze(0))
                    _, predicted = torch.max(output, 1)
                    if(predicted == 0):
                        with open("error.txt", 'a') as f:     
                            f.write(f"0 {x1/w} {y1/h} {x2/w} {y2/h}\n")  # Ghi một dòng tọa độ


Number of raw centers:  31
Dữ liệu đã được ghi vào file test1.txt.


In [13]:
import os
image_folder = "/Users/phamminhtuan/Downloads/Trainning_SET/Images"
label_folder = "/Users/phamminhtuan/Downloads/Trainning_SET/Labels"
test = []
for filename in os.listdir(image_folder):
        image_path = os.path.join(image_folder, filename)
        label_path = os.path.join(label_folder, os.path.splitext(filename)[0] + ".txt")  # Giả định file nhãn cùng tên với file ảnh
        image = cv2.imread(image_path)
        labels, coords = read_file_to_tensors(label_path)
        for i in range(len(coords)):
            if(labels[i] == 0):
                  x1, y1, x2, y2 = find_yolov8_square(image, coords[i])
                  cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        break
cv2.imwrite("/Users/phamminhtuan/Desktop/AIChallenge/avg_coords.jpg", image)


True

In [25]:
def read_tensor_to_tensors(input_tensor):
    """
    Extracts labels and coordinates from an input tensor.

    Args:
        input_tensor (torch.Tensor): Input tensor of shape (N, M), where
            N is the number of lines and M is the number of columns.

    Returns:
        torch.Tensor: A tensor containing all labels (first column of the input tensor).
        torch.Tensor: A tensor containing coordinates (remaining columns of the input tensor).
    """
    # Ensure the input tensor has at least two columns
    

    # Extract the first column as labels
    labels = input_tensor[:, 0]

    # Extract the remaining columns as coordinates
    coords = input_tensor[:, 1]

    return labels, coords

In [31]:
import torch

# Giả sử fixed_circle trả về một danh sách
input_path = "/Users/phamminhtuan/Desktop/AIChallenge/IMG_1581_iter_1.jpg"
coords = fixed_circle(input_path, coord_saver)

# Chuyển đổi danh sách thành tensor

for coord in enumerate(coords):
                    xy_coord = find_yolov8_square(image, coord)
                    x1, y1, x2, y2 = xy_coord
                    input = get_box(image, xy_coord)




Number of raw centers:  32
Dữ liệu đã được ghi vào file test.txt.


ValueError: not enough values to unpack (expected 4, got 2)

In [27]:
print(matrix_coordinate)

[[[(np.float64(0.7137117516629712), np.float64(0.15176134560456997), 0.012860310421286032, np.float64(0.009838146620120596)), (np.float64(0.7306430155210643), np.float64(0.15176134560456997), 0.012860310421286032, np.float64(0.009838146620120596)), (np.float64(0.7475742793791574), np.float64(0.15176134560456997), 0.012860310421286032, np.float64(0.009838146620120596)), (np.float64(0.7645055432372506), np.float64(0.15176134560456997), 0.012860310421286032, np.float64(0.009838146620120596)), (np.float64(0.7814368070953437), np.float64(0.15176134560456997), 0.012860310421286032, np.float64(0.009838146620120596)), (np.float64(0.7983680709534368), np.float64(0.15176134560456997), 0.012860310421286032, np.float64(0.009838146620120596)), (np.float64(0.7137117516629712), np.float64(0.16887972072357982), 0.012860310421286032, np.float64(0.009838146620120596)), (np.float64(0.7306430155210643), np.float64(0.16887972072357982), 0.012860310421286032, np.float64(0.009838146620120596)), (np.float64(0

In [37]:
folder_path = "/Users/phamminhtuan/Downloads/testset1/images"
coord_saver = "test.txt"
result_txt_path = 'results_test_cnn.txt'
folder_path = "/Users/phamminhtuan/Downloads/testset1/images"
labels, coords = read_file_to_tensors(coord_saver)
picked_coords = []
for coord in coords:
    picked_coords.append(coord)
#for i, coord in enumerate(coords):

picked_coords

                    
                    
                    

[tensor([0.7145, 0.1540, 0.0129, 0.0097]),
 tensor([0.7314, 0.1540, 0.0129, 0.0097]),
 tensor([0.7483, 0.1540, 0.0129, 0.0097]),
 tensor([0.7652, 0.1540, 0.0129, 0.0097]),
 tensor([0.7820, 0.1540, 0.0129, 0.0097]),
 tensor([0.7989, 0.1540, 0.0129, 0.0097]),
 tensor([0.7145, 0.1709, 0.0129, 0.0097]),
 tensor([0.7314, 0.1709, 0.0129, 0.0097]),
 tensor([0.7483, 0.1709, 0.0129, 0.0097]),
 tensor([0.7652, 0.1709, 0.0129, 0.0097]),
 tensor([0.7820, 0.1709, 0.0129, 0.0097]),
 tensor([0.7989, 0.1709, 0.0129, 0.0097]),
 tensor([0.7145, 0.1877, 0.0129, 0.0097]),
 tensor([0.7314, 0.1877, 0.0129, 0.0097]),
 tensor([0.7483, 0.1877, 0.0129, 0.0097]),
 tensor([0.7652, 0.1877, 0.0129, 0.0097]),
 tensor([0.7820, 0.1877, 0.0129, 0.0097]),
 tensor([0.7989, 0.1877, 0.0129, 0.0097]),
 tensor([0.7145, 0.2046, 0.0129, 0.0097]),
 tensor([0.7314, 0.2046, 0.0129, 0.0097]),
 tensor([0.7483, 0.2046, 0.0129, 0.0097]),
 tensor([0.7652, 0.2046, 0.0129, 0.0097]),
 tensor([0.7820, 0.2046, 0.0129, 0.0097]),
 tensor([0.